# Training a model with our Own Satellite Image dataset

## Importing Libraries

In [1]:
import os
import glob
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
import cv2
!pip install imagehash
import imagehash

import torchvision.models.segmentation
import torch
import torchdata
import torchvision.transforms as tf

from sklearn.model_selection import train_test_split
from PIL import Image

from pickle import dump

# Check if GPU parallel computing is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [2]:
%pip install geedim -q
%pip install torchgeo -q
%pip install rioxarray -q

In [3]:
import rioxarray

## Mounting Google Drive

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from pathlib import Path
from torchgeo.datasets import RasterDataset, unbind_samples, stack_samples
from torchgeo.samplers import RandomBatchGeoSampler, RandomGeoSampler, Units
from torch.utils.data import DataLoader

In [6]:
# Get the current working directory
current_dir = os.getcwd()

# Print the current working directory
current_dir

'/content'

## Accomodate Paths

In [7]:
# Get the path to the directory
initial_path = "/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data"
dataset_path = "/binary_classification_split/"

# List the contents of the directory
contents = os.listdir(initial_path + dataset_path)

# Print the contents of the directory
for item in contents:
  print(item)

valid_masks
test_landcovers
test_masks
train_masks
train_landcovers
valid_landcovers


## Check Items

In [8]:
test_landcovers_path = initial_path + dataset_path + "/test_landcovers/"

# Contents of the directory
test_landcovers_contents = os.listdir(test_landcovers_path)

print("Test Landcovers items:")
print(len(test_landcovers_contents))

Test Landcovers items:
49


In [9]:
test_masks_path = initial_path + dataset_path + "/test_masks/"

# Contents of the directory
test_masks_contents = os.listdir(test_masks_path)

print("Test Masks items:")
print(len(test_masks_contents))

Test Masks items:
49


In [10]:
train_landcovers_path = initial_path + dataset_path + "/train_landcovers/"

# Contents of the directory
train_landcovers_contents = os.listdir(train_landcovers_path)

print("Train Landcovers items:")
print(len(train_landcovers_contents))

Train Landcovers items:
161


In [11]:
train_masks_path = initial_path + dataset_path + "/train_masks/"

# Contents of the directory
train_masks_contents = os.listdir(train_masks_path)

print("Train Masks items:")
print(len(train_masks_contents))

Train Masks items:
161


In [12]:
valid_landcovers_path = initial_path + dataset_path + "/valid_landcovers/"

# Contents of the directory
valid_landcovers_contents = os.listdir(valid_landcovers_path)

print("Valid Landcovers items:")
print(len(valid_landcovers_contents))

Valid Landcovers items:
21


In [13]:
valid_masks_path = initial_path + dataset_path + "/valid_masks/"

# Contents of the directory
valid_masks_contents = os.listdir(valid_masks_path)

print("Valid Masks items:")
print(len(valid_masks_contents))

Valid Masks items:
21


## This scale function is not working properly (not sure why)

Inspired by:
https://www.geocorner.net/post/artificial-intelligence-for-geospatial-analysis-with-pytorch-s-torchgeo-part-1

In [14]:
# THIS IS NOT WORKING
# We are passing a transform function to the images dataset to scale correctly the values to reflectance values (division by 10,000).
# It only applies to landcovers (not Masks)
# def scale(item: dict):
#     item['image'] = item['image'] / 10000
#     return item

## Creating RasterDatasets for Satellite Images

Inspired by: https://www.geocorner.net/post/artificial-intelligence-for-geospatial-analysis-with-pytorch-s-torchgeo-part-1

In [15]:
from pathlib import Path
root = Path(test_landcovers_path)
assert root.exists()
print(root.as_posix())
test_landcovers = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)

root = Path(test_masks_path)
assert root.exists()
print(root.as_posix())
test_mask = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)
test_mask.is_image = False

from pathlib import Path
root = Path(train_landcovers_path)
assert root.exists()
print(root.as_posix())
train_landcovers = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)

root = Path(train_masks_path)
assert root.exists()
print(root.as_posix())
train_mask = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)
train_mask.is_image = False

from pathlib import Path
root = Path(valid_landcovers_path)
assert root.exists()
print(root.as_posix())
valid_landcovers = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)

root = Path(valid_masks_path)
assert root.exists()
print(root.as_posix())
valid_mask = RasterDataset(paths=root.as_posix(),
                                crs='epsg:3395',
                                res=10,
                                transforms=None)
valid_mask.is_image = False

/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/binary_classification_split/test_landcovers
/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/binary_classification_split/test_masks
/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/binary_classification_split/train_landcovers
/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/binary_classification_split/train_masks
/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Project/Satellite Images/With our own Data/binary_classification_split/valid_landcovers
/content/drive/MyDrive/_UPC/Postgraduate in Artificial Intelligence with Deep Learning/X-Final Pr

## Combining Landcovers with Masks by using torchgeo power

In [16]:
train_dset = train_landcovers & train_mask
valid_dset = valid_landcovers & valid_mask
test_dset = test_landcovers & test_mask

## Defining the Hyper-parameters

In [17]:
# Hyperparameters
hparams = {
    'batch_size': 8,
    'num_epochs': 30,
    'test_batch_size': 8,
    'learning_rate': 0.01,
    'weight_decay': 1e-5,
    'log_interval': 100,
}

## Generating samplers

### RandomGeoSampler

https://torchgeo.readthedocs.io/en/latest/api/samplers.html#torchgeo.samplers.RandomGeoSampler.__init__

Parameters:


*   dataset (GeoDataset) – dataset to index from
*   size (Union[tuple[float, float], float]) – dimensions of each patch
*   length (Optional[int]) – number of random samples to draw per epoch (defaults to approximately the maximal number of non-overlapping chips of size size that could be sampled from the dataset)
*   roi (Optional[BoundingBox]) – region of interest to sample from (minx, maxx, miny, maxy, mint, maxt) (defaults to the bounds of dataset.index)
*   units (Units) – defines if size is in pixel or CRS units


In [18]:
train_sampler = RandomGeoSampler(train_landcovers, size=512, length=160, units=Units.PIXELS)
valid_sampler = RandomGeoSampler(valid_landcovers, size=512, length=20, units=Units.PIXELS)
test_sampler = RandomGeoSampler(test_landcovers, size=512, length=50, units=Units.PIXELS)

## Dataloaders attempt

In [19]:
train_dataloader = DataLoader(train_dset, sampler=train_sampler, batch_size=hparams['batch_size'], collate_fn=stack_samples)
valid_dataloader = DataLoader(valid_dset, sampler=valid_sampler, batch_size=hparams['batch_size'], collate_fn=stack_samples)
test_dataloader = DataLoader(test_dset, sampler=test_sampler, batch_size=hparams['batch_size'], collate_fn=stack_samples)

train_batch = next(iter(train_dataloader))
valid_batch = next(iter(valid_dataloader))
test_batch = next(iter(test_dataloader))
train_batch.keys(), valid_batch.keys()

(dict_keys(['crs', 'bbox', 'image', 'mask']),
 dict_keys(['crs', 'bbox', 'image', 'mask']))

## Batch Visualization

The function plot_batch will check automatically the number of items in the batch and if there are masks associated to arrange the output grid accordingly.

In [20]:
from typing import Iterable, List


def plot_imgs(images: Iterable, axs: Iterable, chnls: List[int] = [2, 1, 0], bright: float = 3.):
  for img, ax in zip(images, axs):
    image = img
    print(image.shape)
    # print(image)
    # Replace the tensor with your actual geotiff tensor
    geotiff_tensor = image.permute(1, 2, 0)

    # Check the shape of the tensor
    height, width, bands = geotiff_tensor.shape

    # Assuming it's an RGB image
    if bands == 3:
      # Normalize pixel values to the range [0, 1]
      normalized_tensor = geotiff_tensor / 255.0

      # Reshape the tensor to (height, width, bands) for plotting
      reshaped_tensor = normalized_tensor.reshape((height, width, bands))

      # Display the image using matplotlib
      ax.imshow(reshaped_tensor)
      ax.axis('off')  # Turn off axis labels

    else:
      print("The tensor does not represent an RGB image. Visualization not supported.")


def plot_msks(masks: Iterable, axs: Iterable):
    for mask, ax in zip(masks, axs):
        print(mask.shape)
        # ax.imshow(mask.squeeze().numpy(), cmap='Blues')
        ax.imshow(mask.squeeze().numpy())
        ax.axis('off')


def plot_batch(batch: dict, bright: float = 3., cols: int = 4, width: int = 5, chnls: List[int] = [2, 1, 0]):

    # Get the samples and the number of items in the batch
    samples = unbind_samples(batch.copy())

    # if batch contains images and masks, the number of images will be doubled
    n = 2 * len(samples) if ('image' in batch) and ('mask' in batch) else len(samples)

    # calculate the number of rows in the grid
    rows = n//cols + (1 if n%cols != 0 else 0)

    # create a grid
    _, axs = plt.subplots(rows, cols, figsize=(cols*width, rows*width))

    if ('image' in batch) and ('mask' in batch):
        # plot the images on the even axis
        plot_imgs(images=map(lambda x: x['image'], samples), axs=axs.reshape(-1)[::2], chnls=chnls, bright=bright) #type: ignore

        # plot the masks on the odd axis
        plot_msks(masks=map(lambda x: x['mask'], samples), axs=axs.reshape(-1)[1::2]) #type: ignore

    else:

        if 'image' in batch:
            plot_imgs(images=map(lambda x: x['image'], samples), axs=axs.reshape(-1), chnls=chnls, bright=bright) #type: ignore

        elif 'mask' in batch:
            plot_msks(masks=map(lambda x: x['mask'], samples), axs=axs.reshape(-1)) #type: ignore

In [21]:
plot_batch(train_batch)

Output hidden; open in https://colab.research.google.com to view.

## Data Normalization (Standardization)

Normally, machine learning methods (deep learning included) benefit from feature scaling. That means standard deviation around 1 and zero mean, by applying the following formula (actually normalization is different from standardization, but I will leave the explanation to the reader: https://www.naukri.com/learning/articles/normalization-and-standardization/)

To do that, we need to first find the mean and standard deviation for each one of the 6s channels in the dataset.

Let’s define a function calculate these statistics and write its results in the variables mean and std. We will use our previously installed rasterio package to open the images and perform a simple average over the statistics for each batch/channel. For the standard deviation, this method is an approximation. For a more precise calculation, please refer to: http://notmatthancock.github.io/2017/03/23/simple-batch-stat-updates.htm.

In [22]:
import rasterio as rio

def calc_statistics(dset: RasterDataset):
        """
        Calculate the statistics (mean and std) for the entire dataset
        Warning: This is an approximation. The correct value should take into account the
        mean for the whole dataset for computing individual stds.
        For correctness I suggest checking: http://notmatthancock.github.io/2017/03/23/simple-batch-stat-updates.html
        """

        # To avoid loading the entire dataset in memory, we will loop through each img
        # The filenames will be retrieved from the dataset's rtree index
        files = [item.object for item in dset.index.intersection(dset.index.bounds, objects=True)]

        # Reseting statistics
        accum_mean = 0
        accum_std = 0

        for file in files:
            img = rio.open(file).read()/10000 #type: ignore
            accum_mean += img.reshape((img.shape[0], -1)).mean(axis=1)
            accum_std += img.reshape((img.shape[0], -1)).std(axis=1)

        # at the end, we shall have 2 vectors with lenght n=chnls
        # we will average them considering the number of images
        return accum_mean / len(files), accum_std / len(files)

print("Test Landcovers - Image Normalization")
mean, std = calc_statistics(test_landcovers)
print(mean, std)
print()
print("Valid Landcovers - Image Normalization")
mean, std = calc_statistics(valid_landcovers)
print(mean, std)
print()
print("Train Landcovers - Image Normalization")
mean, std = calc_statistics(train_landcovers)
print(mean, std)
print()
print("Mean: " + str(mean))
print("Std: " + str(std))

Test Landcovers - Image Normalization
[0.00657766 0.00804456 0.0075479 ] [0.00338505 0.00294967 0.00194538]

Valid Landcovers - Image Normalization
[0.00672649 0.00835924 0.00780101] [0.00335708 0.00281504 0.00189687]

Train Landcovers - Image Normalization
[0.00642393 0.00799009 0.00753127] [0.00329464 0.00282698 0.0018531 ]

Mean: [0.00642393 0.00799009 0.00753127]
Std: [0.00329464 0.00282698 0.0018531 ]


## MyNormalize

https://www.geocorner.net/post/artificial-intelligence-for-geospatial-analysis-with-pytorch-s-torchgeo-part-2

Here we have 6 values in each list. Now we have to use these values to normalize the values every time a batch is created by the dataloader and passed to the trainer. Additionally, if we want to visualize this batch we need to “revert” the standardization, otherwise the true color will not be correct. We will then create a class that will do the trick. We are going to inherit it from the torch.nn.Module class, and define the forward method and also the revert method to “undo” the normalization.


In [23]:
class MyNormalize(torch.nn.Module):
    def __init__(self, mean: List[float], stdev: List[float]):
        super().__init__()

        self.mean = torch.Tensor(mean)[:, None, None]
        self.std = torch.Tensor(stdev)[:, None, None]

    def forward(self, inputs: dict):

        x = inputs["image"][..., : len(self.mean), :, :]

        # if batch
        if inputs["image"].ndim == 4:
            x = (x - self.mean[None, ...]) / self.std[None, ...]

        else:
            x = (x - self.mean) / self.std

        inputs["image"][..., : len(self.mean), :, :] = x

        return inputs

    def revert(self, inputs: dict):
        """
        De-normalize the batch.
        Args:
            inputs (dict): Dictionary with the 'image' key
        """

        x = inputs["image"][..., : len(self.mean), :, :]

        # if batch
        if x.ndim == 4:
            x = inputs["image"][:, : len(self.mean), ...]
            x = x * self.std[None, ...] + self.mean[None, ...]
        else:
            x = x * self.std + self.mean

        inputs["image"][..., : len(self.mean), :, :] = x

        return inputs

## MyNormalize - instantiated

Once the class is defined, we can instantiate it with the mean and std values obtained from our dataset and test the forward pass and the revert pass (code output has been suppressed).

In [24]:
# THIS IS NOT WORKING AS EXPECTED NEITHER
# By using train_batch which has larger length
normalize = MyNormalize(mean=mean, stdev=std)
norm_batch = normalize(train_batch)
plot_batch(norm_batch)

batch = normalize.revert(norm_batch)
plot_batch(batch)

Output hidden; open in https://colab.research.google.com to view.

# Torchgeo Models

https://torchgeo.readthedocs.io/en/stable/api/models.html#torchgeo.models.resnet18

In [25]:
!pip install torchgeo lightly

In [26]:
!pip install timm==0.9.2

In [76]:
import timm
from torchgeo.models import ResNet50_Weights

weights = ResNet50_Weights.SENTINEL2_ALL_MOCO
model = timm.create_model("resnet50", in_chans=weights.meta["in_chans"], num_classes=2)

In [77]:
import tempfile

accelerator = "gpu" if torch.cuda.is_available() else "cpu"
default_root_dir = os.path.join(tempfile.gettempdir(), "experiments")

In [78]:
from lightning.pytorch import Trainer

trainer = Trainer(
    accelerator=accelerator,
    default_root_dir=default_root_dir,
    fast_dev_run=False,
    log_every_n_steps=1,
    min_epochs=1,
    max_epochs=10,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [79]:
model

ResNet(
  (conv1): Conv2d(13, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [96]:
from torchvision.models.segmentation import deeplabv3_resnet50
# from torchvision.models.segmentation import DeepLabV3_ResNet50_Weights

In [97]:
model = deeplabv3_resnet50(weights=None, num_classes=2)

model

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [98]:
import torch.nn as nn

backbone = model.get_submodule('backbone')

conv = nn.modules.conv.Conv2d(
    in_channels=3,
    out_channels=64,
    kernel_size=(7, 7),
    stride=(2, 2),
    padding=(3, 3),
    bias=False
)
backbone.register_module('conv1', conv)

pred = model(torch.randn(3, 3, 512, 512))
pred['out'].shape

torch.Size([3, 2, 512, 512])

In [99]:
from typing import Iterable, List, Callable, Optional

def train_loop(
    epochs: int,
    train_dl: DataLoader,
    val_dl: Optional[DataLoader],
    model: nn.Module,
    loss_fn: Callable,
    optimizer: torch.optim.Optimizer,
    acc_fns: Optional[List]=None,
    batch_tfms: Optional[Callable]=None
):
    # size = len(dataloader.dataset)
    cuda_model = model.cuda()

    for epoch in range(epochs):
        accum_loss = 0
        for batch in train_dl:

            if batch_tfms is not None:
                batch = batch_tfms(batch)

            X = batch['image'].cuda()
            y = batch['mask'].type(torch.long).cuda()
            pred = cuda_model(X)['out']
            loss = loss_fn(pred, y)

            # BackProp
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update the accum loss
            accum_loss += float(loss) / len(train_dl)

        # Testing against the validation dataset
        if acc_fns is not None and val_dl is not None:
            # reset the accuracies metrics
            acc = [0.] * len(acc_fns)

            with torch.no_grad():
                for batch in val_dl:

                    if batch_tfms is not None:
                        batch = batch_tfms(batch)

                    X = batch['image'].type(torch.float32).cuda()
                    y = batch['mask'].type(torch.long).cuda()

                    pred = cuda_model(X)['out']

                    for i, acc_fn in enumerate(acc_fns):
                        acc[i] = float(acc[i] + acc_fn(pred, y)/len(val_dl))

            # at the end of the epoch, print the errors, etc.
            print(f'Epoch {epoch}: Train Loss={accum_loss:.5f} - Accs={[round(a, 3) for a in acc]}')
        else:

            print(f'Epoch {epoch}: Train Loss={accum_loss:.5f}')

In [100]:
from sklearn.metrics import jaccard_score

def oa(pred, y):
    flat_y = y.squeeze()
    flat_pred = pred.argmax(dim=1)
    acc = torch.count_nonzero(flat_y == flat_pred) / torch.numel(flat_y)
    return acc

def iou(pred, y):
    flat_y = y.cpu().numpy().squeeze()
    flat_pred = pred.argmax(dim=1).detach().cpu().numpy()
    return jaccard_score(flat_y.reshape(-1), flat_pred.reshape(-1), zero_division=1.)

def loss(p, t):
    return torch.nn.functional.cross_entropy(p, t.squeeze())


In [101]:
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['learning_rate'], weight_decay=hparams['weight_decay'])
train_loop(hparams['num_epochs'], train_dataloader, valid_dataloader, model, loss, optimizer, acc_fns=[oa, iou])

Epoch 0: Train Loss=0.85751 - Accs=[0.514, 0.009]
Epoch 1: Train Loss=0.68102 - Accs=[0.577, 0.0]
Epoch 2: Train Loss=0.67359 - Accs=[0.602, 0.0]
Epoch 3: Train Loss=0.72635 - Accs=[0.602, 0.0]
Epoch 4: Train Loss=0.69472 - Accs=[0.707, 0.0]
Epoch 5: Train Loss=0.67281 - Accs=[0.589, 0.284]
Epoch 6: Train Loss=0.66357 - Accs=[0.839, 0.679]
Epoch 7: Train Loss=0.65051 - Accs=[0.523, 0.25]
Epoch 8: Train Loss=0.64286 - Accs=[0.613, 0.266]
Epoch 9: Train Loss=0.65576 - Accs=[0.69, 0.441]
Epoch 10: Train Loss=0.68486 - Accs=[0.624, 0.327]
Epoch 11: Train Loss=0.60241 - Accs=[0.715, 0.496]
Epoch 12: Train Loss=0.67743 - Accs=[0.434, 0.341]
Epoch 13: Train Loss=0.71112 - Accs=[0.659, 0.52]
Epoch 14: Train Loss=0.66605 - Accs=[0.484, 0.258]
Epoch 15: Train Loss=0.68843 - Accs=[0.755, 0.415]
Epoch 16: Train Loss=0.67100 - Accs=[0.757, 0.481]
Epoch 17: Train Loss=0.59414 - Accs=[0.562, 0.141]
Epoch 18: Train Loss=0.67157 - Accs=[0.65, 0.433]
Epoch 19: Train Loss=0.62007 - Accs=[0.456, 0.174]
Ep